# Tutoriel : Classification des sols par Deep Learning

Ce tutoriel présente les bases de la classification d'images aériennes pour l'analyse de l'occupation des sols. Nous allons utiliser des méthodes de deep learning pour identifier automatiquement différents types de surfaces (cultures, forêts, surfaces artificielles, etc.) à partir d'images aériennes.

## Objectif
Construire et entraîner un réseau de neurones simple capable de classifier chaque pixel d'une image aérienne selon le type de sol qu'il représente.

## 1. Importation des bibliothèques

Nous commençons par importer les bibliothèques nécessaires :
- **matplotlib** : pour la visualisation des images et des graphiques
- **rasterio** : pour la lecture des images géospatiales
- **torch** : le framework de deep learning que nous utiliserons
- **numpy** : pour les calculs numériques
- **sklearn** : pour les métriques d'évaluation

In [ ]:
import matplotlib.pyplot as plt
import rasterio
import glob
import warnings
import torch 
import numpy as np 
import copy
import sklearn.metrics

warnings.filterwarnings('ignore')

## 2. Sélection des zones géographiques

Le jeu de données contient des images de 10 zones géographiques différentes en France. Pour entraîner notre modèle de manière robuste, nous divisons ces zones en trois ensembles :

- **Zone d'entraînement** : utilisée pour apprendre les paramètres du modèle
- **Zone de validation** : utilisée pour ajuster les hyperparamètres et éviter le surapprentissage
- **Zone de test** : utilisée pour évaluer les performances finales sur des données jamais vues

**Zones disponibles** : Cairanne_84, Chateauroux_36, Chissey_71, Claveyson_26, Fessenheim_68, MaelPestivien_22, SaintCyr_69, SaintHilaire_61, SaintMartin_15, Sauvagnon_64

In [ ]:
zone_entrainement = ''
zone_validation = ''
zone_test = ''

## 3. Définition des classes et des couleurs

### Classes d'occupation du sol

Les annotations originales comportent **19 classes** détaillées. Pour simplifier le problème dans ce tutoriel, nous les regroupons en **5 classes principales** :

0. **Bâtiments** (rouge)
1. **Sol** (jaune) - perméable, imperméable, nu, labouré, ...
2. **Surfaces en eau** (bleu) - rivières, lacs, étangs, piscines, ...
3. **Végétation** (vert) - forêts, haies, arbres, pelouse
4. **Autres/Non-annoté** (noir) - ombres, ...

Chaque classe est associée à une couleur RGB pour faciliter la visualisation des résultats.

In [ ]:
# Correspondance entre les 19 classes d'origine et les 5 classes simplifiées
correspondance_19_vers_5_classes = torch.tensor([0, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 1, 2, 4, 4, 4, 4, 4, 4])

# Palette de couleurs pour les 19 classes d'origine
couleurs_rvb_19_classes = torch.tensor([[219, 14, 154], [147, 142, 123], [248, 12, 0], [169, 113, 1], 
                                         [21, 83, 174], [25, 74, 38], [70, 228, 131], [243, 166, 13], 
                                         [102, 0, 130], [85, 255, 0], [255, 243, 13], [228, 223, 124], 
                                         [61, 230, 235], [255, 255, 255], [138, 179, 160], [107, 113, 79], 
                                         [197, 220, 66], [153, 153, 255], [0, 0, 0]])

# Palette de couleurs pour les 5 classes simplifiées
couleurs_rvb_5_classes = torch.tensor([[255, 65, 54],     # Rouge - Bâtiments
                                        [241, 196, 15],   # Jaune - Sol
                                        [52, 152, 219],   # Bleu - Surfaces en eau
                                        [46, 204, 113],   # Vert - Végétation
                                        [0, 0, 0]])       # Noir - Autres/Non-annoté

## 4. Chargement des données

### Structure des données

Pour chaque zone géographique, nous disposons de :
- **3 images aériennes** (format JPG) de 512×512 pixels en couleur RGB
- **3 masques d'annotation** (format TIF) correspondants, où chaque pixel est étiqueté avec sa classe

La fonction `charger_images()` charge les données d'une zone et effectue automatiquement le regroupement des 19 classes originales vers les 5 classes simplifiées.

In [ ]:
def charger_images(zone):
    """
    Charge les images et annotations d'une zone géographique donnée.
    
    Retourne :
    - images : tenseur des images RGB
    - annotations_19 : annotations en 19 classes
    - annotations_5 : annotations regroupées en 5 classes
    """
    chemins_images = glob.glob(f"sample/{zone}/IMG_*.jpg")
    chemins_annotations = [chemin.replace('IMG', 'MSK').replace('jpg', 'tif') for chemin in chemins_images]
    
    images = torch.stack([torch.tensor(rasterio.open(image).read()).float() for image in chemins_images])
    annotations_19 = torch.stack([torch.tensor(rasterio.open(annot).read()[0]).int() for annot in chemins_annotations]) - 1
    annotations_5 = correspondance_19_vers_5_classes[annotations_19]
    
    return images, annotations_19, annotations_5

In [ ]:
# Chargement des données pour les trois ensembles
images_entrainement, annotations_19_entrainement, annotations_entrainement = charger_images(zone_entrainement)
images_validation, annotations_19_validation, annotations_validation = charger_images(zone_validation)
images_test, annotations_19_test, annotations_test = charger_images(zone_test)

print(f"Données chargées :")
print(f"  - Entraînement : {images_entrainement.shape[0]} images de {images_entrainement.shape[2]}×{images_entrainement.shape[3]} pixels")
print(f"  - Validation : {images_validation.shape[0]} images de {images_entrainement.shape[2]}×{images_entrainement.shape[3]} pixels")
print(f"  - Test : {images_test.shape[0]} images de {images_entrainement.shape[2]}×{images_entrainement.shape[3]} pixels")

## 5. Visualisation des données

Avant de construire notre modèle, il est essentiel de visualiser les données pour comprendre :
- La diversité des paysages entre les différentes zones
- La qualité des annotations
- La distribution des classes

Pour chaque image, nous affichons :
- **Ligne 1** : les images aériennes RGB originales
- **Ligne 2** : les masques d'annotation colorés selon les 5 classes

In [ ]:
def afficher_images(images, annotations, palette_couleurs, predictions=None):
    """
    Affiche les images, leurs annotations et optionnellement les prédictions du modèle.
    """
    nb_lignes = 2 if predictions is None else 3
    
    for k, (image, annotation) in enumerate(zip(images, annotations)):
        img = image.permute(1, 2, 0).int()
        annotation_coloree = palette_couleurs[annotation]
        
        # Affichage de l'image aérienne
        plt.subplot(nb_lignes, 3, k + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f'Image {k+1}', fontsize=10)
        
        # Affichage de l'annotation
        plt.subplot(nb_lignes, 3, 3 + k + 1)
        plt.imshow(annotation_coloree)
        plt.axis('off')
        plt.title(f'Annotation {k+1}', fontsize=10)
        
        # Affichage de la prédiction si disponible
        if predictions is not None:
            prediction_coloree = palette_couleurs[predictions[k]]
            plt.subplot(nb_lignes, 3, 6 + k + 1)
            plt.imshow(prediction_coloree)
            plt.axis('off')
            plt.title(f'Prédiction {k+1}', fontsize=10)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualisation des données d'entraînement
print("=== Zone d'entraînement ===")
afficher_images(images_entrainement, annotations_entrainement, couleurs_rvb_5_classes)

In [ ]:
# Visualisation des données de validation
print("=== Zone de validation ===")
afficher_images(images_validation, annotations_validation, couleurs_rvb_5_classes)

In [ ]:
# Visualisation des données de test
print("=== Zone de test ===")
afficher_images(images_test, annotations_test, couleurs_rvb_5_classes)

## 6. Construction du modèle

### Architecture du réseau de neurones

Nous utilisons un modèle très simple pour ce tutoriel, composé de deux couches :

1. **Couche de convolution 2D** : analyse l'image en appliquant des filtres qui apprennent à détecter des motifs caractéristiques de chaque classe (textures, couleurs, formes)
   - Entrée : 3 canaux (Rouge, Vert, Bleu)
   - Sortie : 4 canaux (une carte de scores pour chacune des 4 classes non-ignorées)
   - Taille du filtre : 7×7 pixels (permet de capturer le contexte local autour de chaque pixel)

2. **Fonction d'activation ReLU** : introduit de la non-linéarité pour permettre au modèle d'apprendre des relations complexes

### Calcul du nombre de paramètres

Le nombre de paramètres à apprendre est calculé ainsi :
- Poids de la convolution : 3 (canaux d'entrée) × 4 (canaux de sortie) × 7 × 7 (taille du filtre) = 588
- Biais : 4 (un par canal de sortie)
- **Total : 592 paramètres**

In [ ]:
modele = torch.nn.Sequential(
    torch.nn.Conv2d(3, 4, kernel_size=7, padding=3, dilation=1, groups=1, 
                    bias=True, padding_mode='reflect', device='cuda'),
    torch.nn.ReLU()
)

def compter_parametres(modele):
    """Calcule le nombre de paramètres apprenables du modèle."""
    return sum(p.numel() for p in modele.parameters() if p.requires_grad)

nb_parametres = compter_parametres(modele)
print(f"Nombre de paramètres du modèle : {nb_parametres}")
print(f"Vérification du calcul : 3 × 4 × 7 × 7 + 4 = {3 * 4 * 7 * 7 + 4}")

## 7. Normalisation des données

### Pourquoi normaliser ?

La normalisation est une étape cruciale en deep learning. Elle consiste à mettre toutes les images à la même échelle en :
- Soustrayant la moyenne de chaque canal
- Divisant par l'écart-type de chaque canal

**Avantages** :
- Accélère la convergence pendant l'entraînement
- Rend le modèle moins sensible aux variations d'illumination
- Améliore la stabilité numérique

**Important** : nous calculons la moyenne et l'écart-type uniquement sur les ensembles d'entraînement et de validation, puis appliquons ces mêmes statistiques à l'ensemble de test pour éviter toute fuite d'information.

In [ ]:
# Calcul des statistiques sur les données d'entraînement et de validation
moyenne = torch.cat([images_entrainement, images_validation]).mean((0, 2, 3))
ecart_type = torch.cat([images_entrainement, images_validation]).std((0, 2, 3))

print(f"Moyenne par canal RGB : {moyenne}")
print(f"Écart-type par canal RGB : {ecart_type}")

# Application de la normalisation à tous les ensembles
images_entrainement_norm = (images_entrainement - moyenne[..., None, None]) / ecart_type[..., None, None]
images_validation_norm = (images_validation - moyenne[..., None, None]) / ecart_type[..., None, None]
images_test_norm = (images_test - moyenne[..., None, None]) / ecart_type[..., None, None]

print("\nNormalisation appliquée avec succès")

## 8. Entraînement du modèle

### Processus d'apprentissage

L'entraînement consiste à ajuster les 592 paramètres du modèle pour minimiser l'erreur de classification. Le processus se déroule en plusieurs étapes :

1. **Propagation avant** : le modèle prédit les classes pour chaque pixel
2. **Calcul de la perte** : on mesure l'écart entre les prédictions et les vraies annotations
3. **Rétropropagation** : on calcule comment ajuster les paramètres pour réduire l'erreur
4. **Mise à jour** : les paramètres sont ajustés dans la bonne direction

### Composants utilisés

- **Fonction de perte** : Cross-Entropy Loss (mesure la qualité de la classification)
- **Optimiseur** : SGD avec momentum (algorithme qui ajuste les paramètres)
- **Scheduler** : ajuste automatiquement le taux d'apprentissage selon les performances

### Métriques suivies

- **Précision (accuracy)** : pourcentage de pixels correctement classés
- **Perte (loss)** : mesure de l'erreur globale du modèle

Ces métriques sont calculées à chaque époque sur l'entraînement et tous les 5 époques sur la validation.

In [ ]:
# Transfert des données vers le GPU pour accélérer les calculs
images_entrainement_norm = images_entrainement_norm.to('cuda')
annotations_entrainement = annotations_entrainement.to('cuda')
images_validation_norm = images_validation_norm.to('cuda')
annotations_validation = annotations_validation.to('cuda')

# Configuration de l'entraînement
optimiseur = torch.optim.SGD(modele.parameters(), lr=0.01, momentum=0.9)
planificateur = torch.optim.lr_scheduler.MultiStepLR(optimiseur, milestones=[1500, 1750], gamma=0.1)
critere = torch.nn.CrossEntropyLoss(ignore_index=4)  # Ignore la classe "autre"

# Variables pour suivre l'évolution
precisions_entrainement, precisions_validation = [], []
pertes_entrainement, pertes_validation = [], []
meilleure_precision_val = 0
meilleur_modele = None

In [ ]:
# Boucle d'entraînement sur 100 époques
nb_epoques = 2000
val_frequence = 100

for epoque in range(1, nb_epoques + 1):
    # Phase d'entraînement
    modele.train()
    optimiseur.zero_grad()
    
    sortie = modele(images_entrainement_norm)
    prediction = torch.argmax(sortie, dim=1)
    perte = critere(sortie, annotations_entrainement)
    precision = (prediction == annotations_entrainement).sum() / (512 * 512 * 3) * 100
    
    perte.backward()
    optimiseur.step()
    planificateur.step(precision)
    
    pertes_entrainement.append(float(perte.item()))
    precisions_entrainement.append(float(precision.item()))
    
    # Évaluation sur la validation tous les 5 époques
    if epoque % val_frequence == 0:
        modele.eval()
        with torch.no_grad():
            sortie_val = modele(images_validation_norm)
            prediction_val = torch.argmax(sortie_val, dim=1)
            perte_val = float(critere(sortie_val, annotations_validation).item())
            precision_val = float((prediction_val == annotations_validation).sum() / (512 * 512 * 3) * 100)
            
            pertes_validation.append(perte_val)
            precisions_validation.append(precision_val)
            
            # Sauvegarde du meilleur modèle
            if precision_val > meilleure_precision_val:
                meilleure_precision_val = precision_val
                meilleur_modele = copy.deepcopy(modele)
        
        print(f"Époque {epoque}/{nb_epoques} - "
              f"Perte train: {perte:.4f}, Précision train: {precision:.2f}% - "
              f"Perte val: {perte_val:.4f}, Précision val: {precision_val:.2f}%")

print("\nEntraînement terminé !")

## 9. Visualisation des courbes d'apprentissage

Les courbes d'apprentissage permettent de diagnostiquer la qualité de l'entraînement :

### Courbe de précision
- **Si train et validation augmentent ensemble** : le modèle apprend correctement
- **Si train augmente mais validation stagne** : surapprentissage (le modèle mémorise les données d'entraînement)

### Courbe de perte
- **Si train et validation diminuent ensemble** : convergence normale
- **Si train diminue mais validation augmente** : surapprentissage

Une bonne généralisation se traduit par des courbes de train et validation qui évoluent de manière similaire.

In [ ]:
# Courbe de précision
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(range(1, nb_epoques + 1), precisions_entrainement, label='Entraînement', linewidth=2)
plt.plot(range(val_frequence, nb_epoques + 1, val_frequence), precisions_validation, label='Validation', linewidth=2, marker='o')
plt.xlabel('Époque', fontsize=12)
plt.ylabel('Précision (%)', fontsize=12)
plt.title('Évolution de la précision', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

# Courbe de perte
plt.subplot(1, 2, 2)
plt.plot(range(1, nb_epoques + 1), pertes_entrainement, label='Entraînement', linewidth=2)
plt.plot(range(val_frequence, nb_epoques + 1, val_frequence), pertes_validation, label='Validation', linewidth=2, marker='o')
plt.xlabel('Époque', fontsize=12)
plt.ylabel('Perte', fontsize=12)
plt.title('Évolution de la perte', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Évaluation sur les données de validation

### Matrice de confusion

La matrice de confusion est un outil essentiel pour analyser les performances du modèle. Elle montre :
- **Diagonale** : nombre de pixels correctement classés pour chaque classe
- **Hors diagonale** : confusions entre classes (ex: forêts classées comme prairies)

Cette analyse permet d'identifier quelles classes sont difficiles à distinguer et pourquoi le modèle se trompe.

In [ ]:
def matrice_de_confusion(images, annotations, split):
    
    # Génération des prédictions sur la validation avec le meilleur modèle
    meilleur_modele.eval()
    with torch.no_grad():
        sortie = meilleur_modele(images.to('cuda'))
        predictions = torch.argmax(sortie, dim=1)
    
    # Calcul de la matrice de confusion
    matrice_confusion = sklearn.metrics.confusion_matrix(
        annotations.cpu().flatten().numpy(), 
        predictions.cpu().flatten().numpy(), 
        labels=[0, 1, 2, 3]
    )
    
    # Normalisation de la matrice de confusion par ligne (par classe réelle)
    # Ne pas ajouter 1e-10, laisser les divisions par zéro créer des NaN
    sommes_lignes = matrice_confusion.sum(axis=1)[:, np.newaxis]
    matrice_confusion_norm = np.divide(
        matrice_confusion.astype('float'), 
        sommes_lignes,
        where=sommes_lignes != 0  # Évite le warning de division par zéro
    )
    # Les lignes avec somme = 0 auront des NaN
    matrice_confusion_norm[sommes_lignes.flatten() == 0] = np.nan
    
    noms_classes = ['Bâti', 'Sol', 'Eau', 'Végét']
    
    plt.figure(figsize=(8, 6))
    
    # Créer une copie pour l'affichage où NaN = 0 (pour le colormap)
    matrice_affichage = np.copy(matrice_confusion_norm)
    matrice_affichage[np.isnan(matrice_affichage)] = 0
    
    plt.imshow(matrice_affichage, interpolation='nearest', cmap='Blues')
    plt.title(f'Matrice de confusion ({split})', fontsize=14, fontweight='bold')
    plt.colorbar(label='Nombre de pixels')
    
    # Ajout des étiquettes
    tick_marks = np.arange(len(noms_classes))
    plt.xticks(tick_marks, noms_classes, rotation=45, ha='right')
    plt.yticks(tick_marks, noms_classes)
    
    # Affichage des valeurs dans chaque case
    for i in range(len(noms_classes)):
        for j in range(len(noms_classes)):
            if np.isnan(matrice_confusion_norm[i, j]):
                # Case grisée pour NaN
                plt.gca().add_patch(plt.Rectangle((j-0.5, i-0.5), 1, 1, 
                                                   fill=True, color='lightgray', 
                                                   edgecolor='white', linewidth=2))
                plt.text(j, i, 'N/A', ha="center", va="center", 
                        color='darkgray', fontsize=12, fontweight='bold')
            else:
                couleur_texte = 'white' if matrice_confusion_norm[i, j] > np.nanmax(matrice_confusion_norm) / 2 else 'black'
                plt.text(j, i, format(matrice_confusion_norm[i, j]*100, '.1f'),
                        ha="center", va="center", color=couleur_texte, fontsize=12, fontweight='bold')
    
    plt.ylabel('Vraie classe', fontsize=12)
    plt.xlabel('Classe prédite', fontsize=12)
    plt.tight_layout()
    plt.show()
    return predictions
    

predictions_entrainement = matrice_de_confusion(images_entrainement_norm, annotations_entrainement, "Entraînement")
predictions_validation = matrice_de_confusion(images_validation_norm, annotations_validation, "Validation")


## 11. Visualisation des prédictions sur les données de test

Nous évaluons maintenant le modèle sur l'ensemble de test, composé d'images d'une zone géographique jamais vue pendant l'entraînement. Cela permet de mesurer la capacité de **généralisation** du modèle.

Pour chaque image, nous affichons :
- **Ligne 1** : l'image aérienne originale
- **Ligne 2** : l'annotation de référence (vérité terrain)
- **Ligne 3** : la prédiction du modèle

Comparer visuellement ces trois lignes permet d'identifier les forces et faiblesses du modèle.

In [ ]:
# Génération des prédictions sur le jeu d'entraînement
meilleur_modele.eval()
with torch.no_grad():
    sortie_entrainement = modele(images_entrainement_norm.to('cuda'))
    predictions_entrainement = torch.argmax(sortie_entrainement, dim=1)

# Génération des prédictions sur le jeu de test
meilleur_modele.eval()
with torch.no_grad():
    sortie_test = modele(images_test_norm.to('cuda'))
    predictions_test = torch.argmax(sortie_test, dim=1)


print("=== Résultats sur la zone d'entraînement ===")
afficher_images(images_entrainement, annotations_entrainement.cpu(), couleurs_rvb_5_classes, predictions_entrainement.cpu())

print("=== Résultats sur la zone de test ===")
afficher_images(images_test, annotations_test, couleurs_rvb_5_classes, predictions_test.cpu())

### Matrice de confusion

On peut également afficher la matrice de confusion sur le jeu de test :

In [ ]:
predictions_test = matrice_de_confusion(images_test_norm, annotations_test, "Test")

## 12. Métriques de performance

Pour quantifier objectivement les performances du modèle, nous calculons plusieurs métriques :

### Précision globale (Overall Accuracy)
Pourcentage de pixels correctement classés tous types confondus.

### Précision par classe (Per-class Accuracy)
Pourcentage de pixels correctement classés pour chaque type d'occupation du sol. Permet d'identifier les classes bien reconnues vs les classes problématiques.

### Précision moyenne (Mean Accuracy)
Moyenne des précisions par classe. Cette métrique donne le même poids à toutes les classes, contrairement à la précision globale qui favorise les classes majoritaires.

**Classes** :
- 0 : Bâtiment
- 1 : Sol
- 2 : Surfaces en eau
- 3 : Végétation

In [ ]:
import pandas as pd
from IPython.display import display

def calculer_metriques(annotations, predictions, labels=[0, 1, 2, 3]):
    """
    Calcule les métriques de performance en gérant les classes absentes.
    """
    # Calcul de la matrice de confusion
    matrice_confusion = sklearn.metrics.confusion_matrix(
        annotations.cpu().flatten().numpy() if hasattr(annotations, 'numpy') else annotations.flatten(),
        predictions.cpu().flatten().numpy() if hasattr(predictions, 'cpu') else predictions.flatten().numpy(),
        labels=labels
    )
    
    # Précision globale (accuracy)
    precision_globale = np.trace(matrice_confusion) / np.sum(matrice_confusion) * 100
    
    # Calcul de la précision par classe (recall) en gérant les divisions par zéro
    sommes_lignes = matrice_confusion.sum(axis=1)
    precision_par_classe = np.divide(
        np.diag(matrice_confusion).astype('float'),
        sommes_lignes,
        out=np.full_like(sommes_lignes, np.nan, dtype='float'),
        where=sommes_lignes != 0
    ) * 100
    
    # Précision moyenne (mean accuracy) sur les classes présentes
    precision_moyenne = np.nanmean(precision_par_classe)
    
    return {
        'Précision globale (%)': precision_globale,
        'Précision moyenne (%)': precision_moyenne,
        'Bâtiment (%)': precision_par_classe[0],
        'Sol (%)': precision_par_classe[1],
        'Eau (%)': precision_par_classe[2],
        'Végétation (%)': precision_par_classe[3],
    }

# Calcul des métriques pour chaque ensemble
metriques_train = calculer_metriques(annotations_entrainement, predictions_entrainement)
metriques_val = calculer_metriques(annotations_validation, predictions_validation)
metriques_test = calculer_metriques(annotations_test, predictions_test)

# Création du DataFrame
df_metriques = pd.DataFrame({
    'Entraînement': metriques_train,
    'Validation': metriques_val,
    'Test': metriques_test
}).T

# Formatage du tableau
def formater_cellule(valeur):
    """Formate les valeurs : affiche N/A pour NaN, sinon le pourcentage."""
    if pd.isna(valeur):
        return 'N/A'
    else:
        return f'{valeur:.2f}'

# Application du formatage
df_formate = df_metriques.map(formater_cellule)

try:
    styled_df = df_formate.style.set_properties(**{
        'text-align': 'center',
        'font-size': '12pt',
    }).set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('padding', '8px')]}
    ])
    display(styled_df)
except:
    print(df_formate)

## Conclusion et perspectives

### Ce que nous avons accompli

Dans ce tutoriel, nous avons construit un modèle simple de classification d'images aériennes capable d'identifier automatiquement différents types d'occupation du sol. Bien que l'architecture soit minimaliste (592 paramètres seulement), elle démontre les concepts fondamentaux du deep learning appliqué à la télédétection.

### Limites du modèle actuel

- Architecture très simple : une seule couche de convolution limite la capacité à capturer des motifs complexes
- Petit jeu de données : seulement 3 images par zone
- Contexte spatial limité : le filtre 7×7 ne capte qu'un voisinage très local

### Vers le modèle CoSIA

- L'architecture Swin+UperNet du modèle CoSIA comporte environ 90 million de paramètres
- Le jeu de données d'entraînement de ce modèle représente 2500 km² répartis sur tout le territoire métropolitain avec des prises de vues allant d'avril à novembre